<a id='top'></a>
<div class="list-group" id="list-tab" role="tablist">
<h3 class="list-group-item list-group-item-action active" data-toggle="list"  role="tab" aria-controls="home">Contents</h3>

[0. Introduction](#0)     
[1. Filtering Data](#1)     
    &nbsp;&nbsp;&nbsp;&nbsp;[Exercise A. Using the filter method](#A)   
    &nbsp;&nbsp;&nbsp;&nbsp;[Exercise B. Using the isNotNull method and combined conditions](#B)    
[2. Selecting](#2)     
    &nbsp;&nbsp;&nbsp;&nbsp;[Exercise C. Using the select method](#C)  
[3. Column Data Type Changes](#3)    
    &nbsp;&nbsp;&nbsp;&nbsp;[Exercise D. Using cast to change data types](#D)    
[4. Aggregating](#4)     
    &nbsp;&nbsp;&nbsp;&nbsp;[Exercise E. Creating GroupedData objects and Aggregating](#E)    
    &nbsp;&nbsp;&nbsp;&nbsp;[Exercise F. Grouping and Aggregating](#F)             
[5. Summary](#5)

<a id="0"></a>
# Introduction

This set of lab exercises continues from the Spark DataFrames introduction with a focus on Spark DataFrame methods that can be used for querying.

As you have seen previously, when .sql() is used with a DataFrame a SQL query can be entered as a string. This is great if you are used to working with SQL, but if you are more comfortable using Pandas data frame methods you might have been wishing for something similar! Additionally, where multiple transformations are required one after another it can be useful to have methods that can be 'dotted' one after another in sequence to produce a single new data frame.

This lab considers how the following methods can be used for querying:

**DataFrame methods**

.filter(condition) This will filter rows using the condition you input and will return all columns (alternative to SELECT * FROM DataFrame WHERE condition).

.isNotNull() This is a useful method for creating conditions that require a data entry to be non-null.

.select(columns) This will return just the columns you request (alternative to SELECT columns FROM DataFrame).

.printSchema() This will show you the table schema. Also: .summary() or .describe(). These are useful if you want a quick summary view.

.groupBy() This is used for grouping rows together with summary statistics of each group (usually used with one or more of the summary statistics methods listed below).

.withColumnRenamed() This creates a new data frame with the title of a column renamed. It needs both the existing column title, and the new one as inputs.

.join() This joins two data frames: the one before the dot with the one you give as the first input. You can optionally give two further inputs: on: to specify which columns are to be matched and how: to specify the type of join (the default is an inner join - where a row is from both tables and repeated rows are not included).

**Column methods**

.cast() For changing the data type of a column.

**Summary Stats**

.min() This gives the smallest value.

.max() This gives the largest value.

.avg() The documentations cryptically states that this gives the average... look out for which type of average!

.sum() This gives the total.

.stddev() This gives the sample standard deviation (average spread of the data).





We must first set up and load the dataset:

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
print(spark)

In [3]:
filepath = "./titanic.csv"

titanic = spark.read.csv(filepath, header = True)

titanic.show()

+------+--------+--------------------+------+------+-----+-----+--------+
|pclass|survived|                name|   sex|   age|sibsp|parch|    fare|
+------+--------+--------------------+------+------+-----+-----+--------+
|     1|       1|Allen, Miss. Elis...|female|    29|    0|    0|211.3375|
|     1|       1|Allison, Master. ...|  male|0.9167|    1|    2|151.5500|
|     1|       0|Allison, Miss. He...|female|     2|    1|    2|151.5500|
|     1|       0|Allison, Mr. Huds...|  male|    30|    1|    2|151.5500|
|     1|       0|Allison, Mrs. Hud...|female|    25|    1|    2|151.5500|
|     1|       1| Anderson, Mr. Harry|  male|    48|    0|    0| 26.5500|
|     1|       1|Andrews, Miss. Ko...|female|    63|    1|    0| 77.9583|
|     1|       0|Andrews, Mr. Thom...|  male|    39|    0|    0|  0.0000|
|     1|       1|Appleton, Mrs. Ed...|female|    53|    2|    0| 51.4792|
|     1|       0|Artagaveytia, Mr....|  male|    71|    0|    0| 49.5042|
|     1|       0|Astor, Col. John ...|

<a id="1"></a>
# 1. Filtering Data

We will start by using .filter(condition). 

This will filter rows using the condition you input in the parentheses and will return all columns. 

It is an alternative to using a query like this one: SELECT * FROM DataFrame WHERE condition. So whatever you would have put as your condition is what you would input in the .filter() method.

A condition must produce a True or False (boolean) result. So it could be just a boolean variable, or it could be an expression which returns either true or false.

Expressions can include the following comparison operators:

== "Is equal to"

<> "Is not equal to". Note that it does not accept =! for not equal to.

< "Is less than"

<= "Is less than or equal to"

\> "Is greater than"

\>= "Is greater than or equal to"

We can pass the expression into the filter method in two different ways.

1. As a string: e.g. titanic.filter("fare < 100")

2. As a column of boolean values: e.g. titanic.filter(fare < 100)

In [21]:
#demonstration
condition = "fare < 100"

result = titanic.filter(condition)

result.show(5)

+------+--------+--------------------+------+---+-----+-----+-------+
|pclass|survived|                name|   sex|age|sibsp|parch|   fare|
+------+--------+--------------------+------+---+-----+-----+-------+
|     1|       1| Anderson, Mr. Harry|  male| 48|    0|    0|26.5500|
|     1|       1|Andrews, Miss. Ko...|female| 63|    1|    0|77.9583|
|     1|       0|Andrews, Mr. Thom...|  male| 39|    0|    0| 0.0000|
|     1|       1|Appleton, Mrs. Ed...|female| 53|    2|    0|51.4792|
|     1|       0|Artagaveytia, Mr....|  male| 71|    0|    0|49.5042|
+------+--------+--------------------+------+---+-----+-----+-------+
only showing top 5 rows



In [20]:
#demonstration
condition = titanic.fare < 100

result = titanic.filter(condition)

result.show(5)

+------+--------+--------------------+------+---+-----+-----+-------+
|pclass|survived|                name|   sex|age|sibsp|parch|   fare|
+------+--------+--------------------+------+---+-----+-----+-------+
|     1|       1| Anderson, Mr. Harry|  male| 48|    0|    0|26.5500|
|     1|       1|Andrews, Miss. Ko...|female| 63|    1|    0|77.9583|
|     1|       0|Andrews, Mr. Thom...|  male| 39|    0|    0| 0.0000|
|     1|       1|Appleton, Mrs. Ed...|female| 53|    2|    0|51.4792|
|     1|       0|Artagaveytia, Mr....|  male| 71|    0|    0|49.5042|
+------+--------+--------------------+------+---+-----+-----+-------+
only showing top 5 rows



<a id="A"></a>
### Exercise A. Using the filter method

1. Use .filter() on titanic to show the rows where the passenger survived.

2. Show the rows where the fare was not £151.55.

3. Show the rows where the passenger was 50 or under.

4. Show the rows where the passenger was travelling with 2 or more siblings or spouses.

In [10]:
condition = "survived == 1"
result = titanic.filter(condition)
result.show(5)

+------+--------+--------------------+------+------+-----+-----+--------+
|pclass|survived|                name|   sex|   age|sibsp|parch|    fare|
+------+--------+--------------------+------+------+-----+-----+--------+
|     1|       1|Allen, Miss. Elis...|female|    29|    0|    0|211.3375|
|     1|       1|Allison, Master. ...|  male|0.9167|    1|    2|151.5500|
|     1|       1| Anderson, Mr. Harry|  male|    48|    0|    0| 26.5500|
|     1|       1|Andrews, Miss. Ko...|female|    63|    1|    0| 77.9583|
|     1|       1|Appleton, Mrs. Ed...|female|    53|    2|    0| 51.4792|
+------+--------+--------------------+------+------+-----+-----+--------+
only showing top 5 rows



In [12]:
condition = "fare <> 151.55"
result = titanic.filter(condition)
result.show(5)

+------+--------+--------------------+------+---+-----+-----+--------+
|pclass|survived|                name|   sex|age|sibsp|parch|    fare|
+------+--------+--------------------+------+---+-----+-----+--------+
|     1|       1|Allen, Miss. Elis...|female| 29|    0|    0|211.3375|
|     1|       1| Anderson, Mr. Harry|  male| 48|    0|    0| 26.5500|
|     1|       1|Andrews, Miss. Ko...|female| 63|    1|    0| 77.9583|
|     1|       0|Andrews, Mr. Thom...|  male| 39|    0|    0|  0.0000|
|     1|       1|Appleton, Mrs. Ed...|female| 53|    2|    0| 51.4792|
+------+--------+--------------------+------+---+-----+-----+--------+
only showing top 5 rows



In [23]:
condition = "age <= 50"
result = titanic.filter(condition)
result.show(5)

+------+--------+--------------------+------+------+-----+-----+--------+
|pclass|survived|                name|   sex|   age|sibsp|parch|    fare|
+------+--------+--------------------+------+------+-----+-----+--------+
|     1|       1|Allen, Miss. Elis...|female|    29|    0|    0|211.3375|
|     1|       1|Allison, Master. ...|  male|0.9167|    1|    2|151.5500|
|     1|       0|Allison, Miss. He...|female|     2|    1|    2|151.5500|
|     1|       0|Allison, Mr. Huds...|  male|    30|    1|    2|151.5500|
|     1|       0|Allison, Mrs. Hud...|female|    25|    1|    2|151.5500|
+------+--------+--------------------+------+------+-----+-----+--------+
only showing top 5 rows



In [24]:
condition = "sibsp >= 2"
result = titanic.filter(condition)
result.show(5)

+------+--------+--------------------+------+---+-----+-----+--------+
|pclass|survived|                name|   sex|age|sibsp|parch|    fare|
+------+--------+--------------------+------+---+-----+-----+--------+
|     1|       1|Appleton, Mrs. Ed...|female| 53|    2|    0| 51.4792|
|     1|       1|Brown, Mrs. John ...|female| 59|    2|    0| 51.4792|
|     1|       1|Cornell, Mrs. Rob...|female| 55|    2|    0| 25.7000|
|     1|       1|Fortune, Miss. Al...|female| 24|    3|    2|263.0000|
|     1|       1|Fortune, Miss. Et...|female| 28|    3|    2|263.0000|
+------+--------+--------------------+------+---+-----+-----+--------+
only showing top 5 rows



We can also combine conditions using logical operators:

& and : (AND) Both conditions must be true to give a final true result.

| or : (OR) Either or both conditions must be true to give a final true result. Note that this takes lower precedence in order of operations (an & will be checked first unless brackets force it otherwise).

For example:

In [38]:
#demonstration
#These conditions give a True result if data is present
condition1 = titanic.age==29
condition2 = titanic.pclass==1
condition3 = titanic.survived==1

#If all three of the selected columns meet the criteria the row will be returned.
result = titanic.filter(condition1 & condition2 & condition3)
result.show()

+------+--------+--------------------+------+---+-----+-----+--------+
|pclass|survived|                name|   sex|age|sibsp|parch|    fare|
+------+--------+--------------------+------+---+-----+-----+--------+
|     1|       1|Allen, Miss. Elis...|female| 29|    0|    0|211.3375|
|     1|       1|   Bird, Miss. Ellen|female| 29|    0|    0|221.7792|
+------+--------+--------------------+------+---+-----+-----+--------+



The above example is equivalent to filtering each condition one at a time. We can 'stack' methods by continuing to dot more methods (they will occur in the order you read them).

This gives the same result:

In [39]:
#demonstration
#These conditions give a True result if data is present
condition1 = titanic.age==29
condition2 = titanic.pclass==1
condition3 = titanic.survived==1

#If all three of the selected columns meet the criteria the row will be returned.
result = titanic.filter(condition1).filter(condition2).filter(condition3)
result.show()

+------+--------+--------------------+------+---+-----+-----+--------+
|pclass|survived|                name|   sex|age|sibsp|parch|    fare|
+------+--------+--------------------+------+---+-----+-----+--------+
|     1|       1|Allen, Miss. Elis...|female| 29|    0|    0|211.3375|
|     1|       1|   Bird, Miss. Ellen|female| 29|    0|    0|221.7792|
+------+--------+--------------------+------+---+-----+-----+--------+



The method .isNotNull() can be used after a column is selected and will return a true or false.

This means it can be used as the condition in a filter.

In [47]:
#Demonstration
#First, how many rows in the whole set:
titanic.count()

1309

In [48]:
#Demonstration
#Now filter out the rows that have age missing, followed by counting how many rows are left:
condition = titanic.age.isNotNull()
print(titanic.filter(condition).count())

1046


<a id="B"></a>
### Exercise B. Using the isNotNull method and combined conditions

These questions require you to create conditions using .isNotNull() on columns and either the use of logical operators or method stacking to return combined results.

1. Create conditions to check if age, sibsp, and parch contain data.

2. Use .filter() with logical operators to return all the rows that aren't missing age, sibsp, parch (i.e. no missing values). Use .count() to check your results.

3. Use .filter() stacked three times to produce the same results.

4. Extend your previous answer by stacking further methods to display the results.

5. Did anyone survive third class who were 60 or over?

Discuss: Is it better to have one line of code made up of stacked methods? 

Prompts: Do you think it will run any differently? (faster/slower/more memory?)
Which is easier for you and other developers to read?

In [49]:
condition1 = titanic.age.isNotNull()
condition2 = titanic.sibsp.isNotNull()
condition3 = titanic.parch.isNotNull()

In [50]:
result = titanic.filter(condition1 & condition2 & condition3)
result.show()
result.count()

+------+--------+--------------------+------+------+-----+-----+--------+
|pclass|survived|                name|   sex|   age|sibsp|parch|    fare|
+------+--------+--------------------+------+------+-----+-----+--------+
|     1|       1|Allen, Miss. Elis...|female|    29|    0|    0|211.3375|
|     1|       1|Allison, Master. ...|  male|0.9167|    1|    2|151.5500|
|     1|       0|Allison, Miss. He...|female|     2|    1|    2|151.5500|
|     1|       0|Allison, Mr. Huds...|  male|    30|    1|    2|151.5500|
|     1|       0|Allison, Mrs. Hud...|female|    25|    1|    2|151.5500|
|     1|       1| Anderson, Mr. Harry|  male|    48|    0|    0| 26.5500|
|     1|       1|Andrews, Miss. Ko...|female|    63|    1|    0| 77.9583|
|     1|       0|Andrews, Mr. Thom...|  male|    39|    0|    0|  0.0000|
|     1|       1|Appleton, Mrs. Ed...|female|    53|    2|    0| 51.4792|
|     1|       0|Artagaveytia, Mr....|  male|    71|    0|    0| 49.5042|
|     1|       0|Astor, Col. John ...|

1046

In [51]:
result = titanic.filter(condition1).filter(condition2).filter(condition3)
result.show()
result.count()

+------+--------+--------------------+------+------+-----+-----+--------+
|pclass|survived|                name|   sex|   age|sibsp|parch|    fare|
+------+--------+--------------------+------+------+-----+-----+--------+
|     1|       1|Allen, Miss. Elis...|female|    29|    0|    0|211.3375|
|     1|       1|Allison, Master. ...|  male|0.9167|    1|    2|151.5500|
|     1|       0|Allison, Miss. He...|female|     2|    1|    2|151.5500|
|     1|       0|Allison, Mr. Huds...|  male|    30|    1|    2|151.5500|
|     1|       0|Allison, Mrs. Hud...|female|    25|    1|    2|151.5500|
|     1|       1| Anderson, Mr. Harry|  male|    48|    0|    0| 26.5500|
|     1|       1|Andrews, Miss. Ko...|female|    63|    1|    0| 77.9583|
|     1|       0|Andrews, Mr. Thom...|  male|    39|    0|    0|  0.0000|
|     1|       1|Appleton, Mrs. Ed...|female|    53|    2|    0| 51.4792|
|     1|       0|Artagaveytia, Mr....|  male|    71|    0|    0| 49.5042|
|     1|       0|Astor, Col. John ...|

1046

In [53]:
titanic.filter(condition1).filter(condition2).filter(condition3).show()

+------+--------+--------------------+------+------+-----+-----+--------+
|pclass|survived|                name|   sex|   age|sibsp|parch|    fare|
+------+--------+--------------------+------+------+-----+-----+--------+
|     1|       1|Allen, Miss. Elis...|female|    29|    0|    0|211.3375|
|     1|       1|Allison, Master. ...|  male|0.9167|    1|    2|151.5500|
|     1|       0|Allison, Miss. He...|female|     2|    1|    2|151.5500|
|     1|       0|Allison, Mr. Huds...|  male|    30|    1|    2|151.5500|
|     1|       0|Allison, Mrs. Hud...|female|    25|    1|    2|151.5500|
|     1|       1| Anderson, Mr. Harry|  male|    48|    0|    0| 26.5500|
|     1|       1|Andrews, Miss. Ko...|female|    63|    1|    0| 77.9583|
|     1|       0|Andrews, Mr. Thom...|  male|    39|    0|    0|  0.0000|
|     1|       1|Appleton, Mrs. Ed...|female|    53|    2|    0| 51.4792|
|     1|       0|Artagaveytia, Mr....|  male|    71|    0|    0| 49.5042|
|     1|       0|Astor, Col. John ...|

In [57]:
conditionA = "survived == 1"
conditionB = "pclass == 3"
conditionC = "age >= 60"

titanic.filter(conditionA).filter(conditionB).filter(conditionC).show()

+------+--------+--------------------+------+---+-----+-----+------+
|pclass|survived|                name|   sex|age|sibsp|parch|  fare|
+------+--------+--------------------+------+---+-----+-----+------+
|     3|       1|Turkula, Mrs. (He...|female| 63|    0|    0|9.5875|
+------+--------+--------------------+------+---+-----+-----+------+



<a id="2"></a>
# 2. Selecting

The previous examples using .filter() allowed us how to report particular records (rows), but they still returned all columns in the table.

This could be quite a problem if you have a significant number of columns!

Even if you have a small table it is useful to be able to select particular columns.

.filter(condition) gives us the equivalent of SELECT * FROM table WHERE condition.

.select(columns) gives us the equivalent of SELECT columns FROM table.

During the exercises we will begin by displaying some columns and build up to using this with filter (to produce the equivalent of SELECT columns FROM table WHERE condition).

To use .select() just list the names of your columns seperated by commas as inputs in the parentheses.

To give a column name you can either do this with quotation marks (as a string) or give the name of the data frame dot column.

i.e. either table.select("col1", "col2") or table.select(table.col1, table.col2).


<a id="C"></a>
### Exercise C. Using the select method

1. Use the titanic DataFrame with .select() to display the age column

2. Save the pclass, sex, and age columns as selection1. Display selection1 to check your results.

3. Find another way to do the same thing as the previous question.

4. Display the class and ages of passengers in second class.

5. Display just the ages of passengers in second class. Does it make a difference whether you use .filter() or .select() first?

6. Display the class, sex, and age of female passengers in second class who survived.

Challenge

7. Display the survival, sex, and age of passengers who were either female or were under 18 (either sex). How many women and children survived?

In [7]:
titanic.select("age").show()
#or 
#titanic.select(titanic.age).show()

+------+
|   age|
+------+
|    29|
|0.9167|
|     2|
|    30|
|    25|
|    48|
|    63|
|    39|
|    53|
|    71|
|    47|
|    18|
|    24|
|    26|
|    80|
|  null|
|    24|
|    50|
|    32|
|    36|
+------+
only showing top 20 rows



In [10]:
#2 ways of doing this:
selection1 = titanic.select("pclass", "sex", "age")
#or
#selection1 = titanic.select(titanic.pclass, titanic.sex, titanic.age)
selection1.show()

+------+------+------+
|pclass|   sex|   age|
+------+------+------+
|     1|female|    29|
|     1|  male|0.9167|
|     1|female|     2|
|     1|  male|    30|
|     1|female|    25|
|     1|  male|    48|
|     1|female|    63|
|     1|  male|    39|
|     1|female|    53|
|     1|  male|    71|
|     1|  male|    47|
|     1|female|    18|
|     1|female|    24|
|     1|female|    26|
|     1|  male|    80|
|     1|  male|  null|
|     1|  male|    24|
|     1|female|    50|
|     1|female|    32|
|     1|  male|    36|
+------+------+------+
only showing top 20 rows



In [6]:
condition = "pclass == 2"
titanic.select("pclass", "age").filter(condition).show()

+------+---+
|pclass|age|
+------+---+
|     2| 30|
|     2| 28|
|     2| 30|
|     2| 18|
|     2| 25|
|     2| 34|
|     2| 36|
|     2| 57|
|     2| 18|
|     2| 23|
|     2| 36|
|     2| 28|
|     2| 51|
|     2| 32|
|     2| 19|
|     2| 28|
|     2|  1|
|     2|  4|
|     2| 12|
|     2| 36|
+------+---+
only showing top 20 rows



In [9]:
condition = "pclass == 2"
titanic.select("age").filter(condition).show()

+---+
|age|
+---+
| 30|
| 28|
| 30|
| 18|
| 25|
| 34|
| 36|
| 57|
| 18|
| 23|
| 36|
| 28|
| 51|
| 32|
| 19|
| 28|
|  1|
|  4|
| 12|
| 36|
+---+
only showing top 20 rows



In [10]:
condition = "pclass == 2"
titanic.filter(condition).select("age").show()

+---+
|age|
+---+
| 30|
| 28|
| 30|
| 18|
| 25|
| 34|
| 36|
| 57|
| 18|
| 23|
| 36|
| 28|
| 51|
| 32|
| 19|
| 28|
|  1|
|  4|
| 12|
| 36|
+---+
only showing top 20 rows



In [14]:
condition1 = "pclass == 2"
condition2 = "sex == 'female'"
condition3 = "survived == 1"
titanic.select("pclass", "sex", "age").filter(condition1).filter(condition2).filter(condition3).show()

+------+------+---+
|pclass|   sex|age|
+------+------+---+
|     2|female| 28|
|     2|female| 36|
|     2|female| 36|
|     2|female| 19|
|     2|female|  4|
|     2|female| 12|
|     2|female| 36|
|     2|female| 19|
|     2|female| 24|
|     2|female| 15|
|     2|female| 40|
|     2|female| 20|
|     2|female| 36|
|     2|female| 42|
|     2|female| 22|
|     2|female| 35|
|     2|female| 25|
|     2|female| 45|
|     2|female| 28|
|     2|female|  8|
+------+------+---+
only showing top 20 rows



In [17]:
condition1 = "sex == 'female'"
condition2 = "age < 18"
condition3 = "survived == 1"
survivingWomenAndChildren = titanic.select("survived", "sex", "age").filter(condition1 or condition2).filter(condition3)
survivingWomenAndChildren.show()

+--------+------+---+
|survived|   sex|age|
+--------+------+---+
|       1|female| 29|
|       1|female| 63|
|       1|female| 53|
|       1|female| 18|
|       1|female| 24|
|       1|female| 26|
|       1|female| 50|
|       1|female| 32|
|       1|female| 47|
|       1|female| 42|
|       1|female| 29|
|       1|female| 19|
|       1|female| 35|
|       1|female| 30|
|       1|female| 58|
|       1|female| 45|
|       1|female| 22|
|       1|female| 44|
|       1|female| 59|
|       1|female| 60|
+--------+------+---+
only showing top 20 rows



In [18]:
survivingWomenAndChildren.count()

339

<a id="3"></a>
# 3. Column Data Type Changes

Shortly we will move on to perform calculations and work out summary statistics on particular columns - using .select() to pick out the ones we wish to work with.

For instance, an analyst might ask themselves what the typical age of survivors was, whether having siblings/spouses, parents/children, or being in a particular class changed anything about your survival chances.

All this first requires that the columns are a numeric data type in order for the mathematical operations in our commands to work.

.printSchema() can be used to quickly see the column names and data types.

Below you can see that parch (number of parents or children a passenger has) is a string. In the demonstration you will see .cast() used to convert this to a string and then .printSchema() and .show() to check the results.

In [19]:
#Demonstration
titanic.printSchema()

root
 |-- pclass: string (nullable = true)
 |-- survived: string (nullable = true)
 |-- name: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- age: string (nullable = true)
 |-- sibsp: string (nullable = true)
 |-- parch: string (nullable = true)
 |-- fare: string (nullable = true)



In [20]:
#Demonstration
titanicNumeric = titanic.withColumn("parch", titanic.parch.cast("Int"))
titanicNumeric.show()
titanicNumeric.printSchema()

+------+--------+--------------------+------+------+-----+-----+--------+
|pclass|survived|                name|   sex|   age|sibsp|parch|    fare|
+------+--------+--------------------+------+------+-----+-----+--------+
|     1|       1|Allen, Miss. Elis...|female|    29|    0|    0|211.3375|
|     1|       1|Allison, Master. ...|  male|0.9167|    1|    2|151.5500|
|     1|       0|Allison, Miss. He...|female|     2|    1|    2|151.5500|
|     1|       0|Allison, Mr. Huds...|  male|    30|    1|    2|151.5500|
|     1|       0|Allison, Mrs. Hud...|female|    25|    1|    2|151.5500|
|     1|       1| Anderson, Mr. Harry|  male|    48|    0|    0| 26.5500|
|     1|       1|Andrews, Miss. Ko...|female|    63|    1|    0| 77.9583|
|     1|       0|Andrews, Mr. Thom...|  male|    39|    0|    0|  0.0000|
|     1|       1|Appleton, Mrs. Ed...|female|    53|    2|    0| 51.4792|
|     1|       0|Artagaveytia, Mr....|  male|    71|    0|    0| 49.5042|
|     1|       0|Astor, Col. John ...|

<a id="D"></a>
### Exercise D. Using cast to change data types

1. Change the data types of parch, sibsp, pclass, survived to integers and save this as titanicNumeric. Use .printSchema() to check your work.

2. Using titanicNumeric (instead of titanic) and "Float" (instead of "Int") change the age and fare columns to floats (numbers with decimal places). Use .printSchema()) to check your work.

3. Change one of your numeric columns in titanicNumeric back to a string, check it and change it back again.

4. Change the survived column in titanicNumeric to a boolean (True or False).

In [21]:
titanicNumeric = titanic.withColumn("parch", titanic.parch.cast("Int")).withColumn("sibsp", titanic.sibsp.cast("Int")).withColumn("pclass", titanic.pclass.cast("Int")).withColumn("survived", titanic.survived.cast("Int"))
titanicNumeric.printSchema()

root
 |-- pclass: integer (nullable = true)
 |-- survived: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- age: string (nullable = true)
 |-- sibsp: integer (nullable = true)
 |-- parch: integer (nullable = true)
 |-- fare: string (nullable = true)



In [23]:
titanicNumeric = titanicNumeric.withColumn("age", titanicNumeric.age.cast("Float")).withColumn("fare", titanicNumeric.fare.cast("Float"))
titanicNumeric.printSchema()

root
 |-- pclass: integer (nullable = true)
 |-- survived: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- age: float (nullable = true)
 |-- sibsp: integer (nullable = true)
 |-- parch: integer (nullable = true)
 |-- fare: float (nullable = true)



In [24]:
titanicNumeric = titanicNumeric.withColumn("age", titanicNumeric.age.cast("String"))
titanicNumeric.printSchema()

root
 |-- pclass: integer (nullable = true)
 |-- survived: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- age: string (nullable = true)
 |-- sibsp: integer (nullable = true)
 |-- parch: integer (nullable = true)
 |-- fare: float (nullable = true)



In [25]:
titanicNumeric = titanicNumeric.withColumn("age", titanicNumeric.age.cast("Float"))
titanicNumeric.printSchema()

root
 |-- pclass: integer (nullable = true)
 |-- survived: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- age: float (nullable = true)
 |-- sibsp: integer (nullable = true)
 |-- parch: integer (nullable = true)
 |-- fare: float (nullable = true)



In [26]:
titanicNumeric = titanicNumeric.withColumn("survived", titanicNumeric.survived.cast("Boolean"))
titanicNumeric.show()
titanicNumeric.printSchema()

+------+--------+--------------------+------+------+-----+-----+--------+
|pclass|survived|                name|   sex|   age|sibsp|parch|    fare|
+------+--------+--------------------+------+------+-----+-----+--------+
|     1|    true|Allen, Miss. Elis...|female|  29.0|    0|    0|211.3375|
|     1|    true|Allison, Master. ...|  male|0.9167|    1|    2|  151.55|
|     1|   false|Allison, Miss. He...|female|   2.0|    1|    2|  151.55|
|     1|   false|Allison, Mr. Huds...|  male|  30.0|    1|    2|  151.55|
|     1|   false|Allison, Mrs. Hud...|female|  25.0|    1|    2|  151.55|
|     1|    true| Anderson, Mr. Harry|  male|  48.0|    0|    0|   26.55|
|     1|    true|Andrews, Miss. Ko...|female|  63.0|    1|    0| 77.9583|
|     1|   false|Andrews, Mr. Thom...|  male|  39.0|    0|    0|     0.0|
|     1|    true|Appleton, Mrs. Ed...|female|  53.0|    2|    0| 51.4792|
|     1|   false|Artagaveytia, Mr....|  male|  71.0|    0|    0| 49.5042|
|     1|   false|Astor, Col. John ...|

<a id="4"></a>
# 4. Aggregating

Aggregations (or summary statistics) are accessed by using .groupBy() to create a GroupedData object.

This is a whole new class of objects seperate from DataFrames just so we can group rows to calculate aggregations.

Here is a reminder of the ones we will look at in the subsequent exercises.


.groupBy() This is used for grouping rows together with summary statistics of each group (usually used with one or more of the summary statistics methods listed below).

**Summary Statistics**

These are some of the methods that come with a GroupedData object:

.min() This gives the smallest value.

.max() This gives the largest value.

.avg() The documentations cryptically states that this gives the average... look out for which type of average!

.sum() This gives the total.

.stddev() This gives the sample standard deviation (average spread of the data).

In [27]:
#Demonstration
#This filters to include female passengers, groups the results into a GroupedData object, finds the smallest (minimum) fare, and displays it.
condition = "sex == 'female'"
titanicNumeric.filter(condition).groupBy().min("fare").show()

+---------+
|min(fare)|
+---------+
|     6.75|
+---------+



<a id="E"></a>
### Exercise E. Creating GroupedData objects and Aggregating

1. Find the maximum fare paid by a female, using .max() and display your results.

2. Find the smallest and largest fares paid by males and display your results.

3. Find the average (using .avg()) fare paid by females.

4. What's the total of fares paid by females? Using .sum()

In [28]:
condition = "sex == 'female'"
titanicNumeric.filter(condition).groupBy().max("fare").show()

+---------+
|max(fare)|
+---------+
| 512.3292|
+---------+



In [31]:
condition = "sex == 'male'"
titanicNumeric.filter(condition).groupBy().max("fare").show()
titanicNumeric.filter(condition).groupBy().min("fare").show()

+---------+
|max(fare)|
+---------+
| 512.3292|
+---------+

+---------+
|min(fare)|
+---------+
|      0.0|
+---------+



In [34]:
condition = "sex == 'female'"
titanicNumeric.filter(condition).groupBy().avg("fare").show()
titanicNumeric.filter(condition).groupBy().sum("fare").show()

+------------------+
|         avg(fare)|
+------------------+
|46.198096679515594|
+------------------+

+------------------+
|         sum(fare)|
+------------------+
|21528.313052654266|
+------------------+



 **Grouping and Aggregating**
 
 Up to this point we've used .groupBy() with no inputs. However, if you choose a column name to put in the parentheses you can group rows together first, then work out a summary for each group.
 
 This means we can, for example, quickly compare both female and male passenger results in one step.
 
 In this example we will group by "sex" (so the rows for female passengers and male passengers are separated).
 
 Then we will use .count() to summarise how many rows are in each group.


In [35]:
titanicNumeric.groupBy("sex").count().show()

+------+-----+
|   sex|count|
+------+-----+
|female|  466|
|  male|  843|
+------+-----+



<a id="E"></a>
### Exercise F. Creating GroupedData objects and Aggregating

1. Group the rows in titanicNumeric by sex, calculate the smallest fare, and display your results. Compare your answer to the previous exercise.

2. Repeat the last question for .max(), .avg() and .sum() to compare how fares differed for male and female passengers.

3. Group the rows by pclass and use min, max, and avg to compare the fares for each class.

*Challenge*

4. Group the rows by pclass and use avg to compare the survival rate for each class.

Hints: The survived column is currently boolean and needs to be integer.

Using .avg() we can work out a proportion of those who survived: for example, the average of these 5 passengers: 1, 0, 0, 1, 1 is 0.6 or 60%.

*Explore*

5. How does the survival rate differ for male or female passengers, what about if they have parents/children or siblings/spouses with them? What are the ages of survivors like?

In [36]:
titanicNumeric.groupBy("sex").min("fare").show()

+------+---------+
|   sex|min(fare)|
+------+---------+
|female|     6.75|
|  male|      0.0|
+------+---------+



In [37]:
titanicNumeric.groupBy("sex").max("fare").show()
titanicNumeric.groupBy("sex").avg("fare").show()
titanicNumeric.groupBy("sex").sum("fare").show()

+------+---------+
|   sex|max(fare)|
+------+---------+
|female| 512.3292|
|  male| 512.3292|
+------+---------+

+------+------------------+
|   sex|         avg(fare)|
+------+------------------+
|female|46.198096679515594|
|  male| 26.15460092523036|
+------+------------------+

+------+------------------+
|   sex|         sum(fare)|
+------+------------------+
|female|21528.313052654266|
|  male| 22022.17397904396|
+------+------------------+



In [39]:
titanicNumeric.groupBy("pclass").min("fare").show()
titanicNumeric.groupBy("pclass").max("fare").show()
titanicNumeric.groupBy("pclass").avg("fare").show()

+------+---------+
|pclass|min(fare)|
+------+---------+
|     1|      0.0|
|     3|      0.0|
|     2|      0.0|
+------+---------+

+------+---------+
|pclass|max(fare)|
+------+---------+
|     1| 512.3292|
|     3|    69.55|
|     2|     73.5|
+------+---------+

+------+------------------+
|pclass|         avg(fare)|
+------+------------------+
|     1| 87.50899178472466|
|     3|13.302888809287616|
|     2|21.179196419698666|
+------+------------------+



In [42]:
titanicNumeric = titanicNumeric.withColumn("survived", titanicNumeric.survived.cast("Int"))
titanicNumeric.printSchema()

root
 |-- pclass: integer (nullable = true)
 |-- survived: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- age: float (nullable = true)
 |-- sibsp: integer (nullable = true)
 |-- parch: integer (nullable = true)
 |-- fare: float (nullable = true)



In [43]:
titanicNumeric.groupBy("pclass").avg("survived").show()

+------+------------------+
|pclass|     avg(survived)|
+------+------------------+
|     1|0.6191950464396285|
|     3|0.2552891396332863|
|     2|0.4296028880866426|
+------+------------------+



In [44]:
#How does the survival rate differ for male or female passengers
titanicNumeric.groupBy("sex").avg("survived").show()

+------+-------------------+
|   sex|      avg(survived)|
+------+-------------------+
|female| 0.7274678111587983|
|  male|0.19098457888493475|
+------+-------------------+



In [45]:
#What about if they have parents/children with them?
titanicNumeric.groupBy("parch").avg("survived").show()

+-----+-------------------+
|parch|      avg(survived)|
+-----+-------------------+
|    1| 0.5882352941176471|
|    6|                0.0|
|    3|              0.625|
|    5|0.16666666666666666|
|    9|                0.0|
|    4|0.16666666666666666|
|    2|  0.504424778761062|
|    0|0.33532934131736525|
+-----+-------------------+



In [46]:
#or siblings/spouses with them?
titanicNumeric.groupBy("sibsp").avg("survived").show()

+-----+-------------------+
|sibsp|      avg(survived)|
+-----+-------------------+
|    1| 0.5109717868338558|
|    3|                0.3|
|    5|                0.0|
|    4|0.13636363636363635|
|    8|                0.0|
|    2| 0.4523809523809524|
|    0| 0.3468013468013468|
+-----+-------------------+



In [48]:
#What are the ages of survivors like?
titanicNumeric.groupBy("survived").min("age").show()
titanicNumeric.groupBy("survived").avg("age").show()
titanicNumeric.groupBy("survived").max("age").show()

+--------+--------+
|survived|min(age)|
+--------+--------+
|       1|  0.1667|
|       0|  0.3333|
+--------+--------+

+--------+------------------+
|survived|          avg(age)|
+--------+------------------+
|       1|28.918228103070405|
|       0|30.545368820669577|
+--------+------------------+

+--------+--------+
|survived|max(age)|
+--------+--------+
|       1|    80.0|
|       0|    74.0|
+--------+--------+



**Measuring Spread**

In the last part of the *Explore* solutions there didn't appear to be very much difference between the youngest, oldest, and average aged passenger when it came to survival rate.

There might be a difference in how spread out the data is though on average - especially if we want less emphasis on the youngest age recorded (which does look a bit erroneous!).

This can be measured using standard deviation (.stddev()) - the smaller the value the more consistent or closer the data is with the mean.

To use .stddev() we need to import some additional functions which contains a section on aggregate (agg for short) functions.


We can also use .agg() method to pass an aggregate column expression that uses any of the aggregate functions from the pyspark.sql.functions submodule.

This submodule contains many useful functions for computing things like standard deviations. All the aggregation functions in this submodule take the name of a column in a GroupedData table.

In [50]:
# Demonstration
import pyspark.sql.functions as sqlf

titanicNumeric.groupBy("survived").agg(sqlf.stddev("age")).show()

+--------+------------------+
|survived|  stddev_samp(age)|
+--------+------------------+
|       1|15.061481385024198|
|       0| 13.92253872102884|
+--------+------------------+



<a id="6"></a>
# 6. Summary

In this section we have learnt about how data frames are used in many languages to store tables of data as collections of columns.

We have used SparkSession - a platform that contains SparkContext, within which you can create Spark DataFrames by connecting to data sources in many different ways.

During the exercises we have explored the following methods in particular. Take a moment to summarise each of these for yourself.

**DataFrame methods**

.filter(condition) This will filter rows using the condition you input and will return all columns (alternative to SELECT * FROM DataFrame WHERE condition).

.isNotNull() This is a useful method for creating conditions that require a data entry to be non-null.

.select(columns) This will return just the columns you request (alternative to SELECT columns FROM DataFrame).

.printSchema() This will show you the table schema. Also: .summary() or .describe(). These are useful if you want a quick summary view.

.groupBy() This is used for grouping rows together with summary statistics of each group (usually used with one or more of the summary statistics methods listed below).

.withColumnRenamed() This creates a new data frame with the title of a column renamed. It needs both the existing column title, and the new one as inputs.

.join() This joins two data frames: the one before the dot with the one you give as the first input. You can optionally give two further inputs: on: to specify which columns are to be matched and how: to specify the type of join (the default is an inner join - where a row is from both tables and repeated rows are not included).

**Column methods**

.cast() For changing the data type of a column.

**Summary Stats**

.min() This gives the smallest value.

.max() This gives the largest value.

.avg() The documentations cryptically states that this gives the average... look out for which type of average!

.sum() This gives the total.

.stddev() This gives the sample standard deviation (average spread of the data).

*Further Reading*

Published documentation with method listings on Column, Functions, and Grouping can be found here:

https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/column.html

https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/functions.html

https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/grouping.html
